<a href="https://colab.research.google.com/github/jpbeaud/language/blob/main/essai1_seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install tensorflow
!pip install keras-preprocessing

Préparation des données

In [22]:

from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
import numpy as np

# Exemples de données (à remplacer par tes propres données)
input_texts = ["bonjour", "salut"]
target_texts = ["hello", "hi"]

# Tokenisation des textes
num_encoder_tokens = len(input_texts)
num_decoder_tokens = len(target_texts)
input_tokenizer = Tokenizer(num_words=num_encoder_tokens)
print("num_encorder_tokens = ", num_encoder_tokens)
print("num_decoder_tokens = ", num_decoder_tokens)
input_tokenizer.fit_on_texts(input_texts)
encoder_input_data = input_tokenizer.texts_to_sequences(input_texts)
encoder_input_data = pad_sequences(encoder_input_data, padding='post')

target_tokenizer = Tokenizer(num_words=num_decoder_tokens)
target_tokenizer.fit_on_texts(target_texts)
decoder_input_data = target_tokenizer.texts_to_sequences(target_texts)
decoder_input_data = pad_sequences(decoder_input_data, padding='post')


num_encorder_tokens =  2
num_decoder_tokens =  2


Entrainement du modèle avec les données préparées

In [24]:
# Convertir les données cibles en une représentation à une dimension
decoder_target_data = np.zeros((len(target_texts), max([len(t) for t in decoder_input_data]), num_decoder_tokens))
for i, seqs in enumerate(decoder_input_data):
    for t, token in enumerate(seqs):
        if token > 0:
            decoder_target_data[i, t, token] = 1.0

# Entraîner le modèle
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,batch_size=64,epochs=100,validation_split=0.2)


Epoch 1/100


ValueError: Exception encountered when calling Functional.call().

[1mInvalid input shape for input Tensor("functional_4_1/Cast:0", shape=(None, 1), dtype=float32). Expected shape (None, None, 100), but input has incompatible shape (None, 1)[0m

Arguments received by Functional.call():
  • inputs=('tf.Tensor(shape=(None, 1), dtype=int32)', 'tf.Tensor(shape=(None, 1), dtype=int32)')
  • training=True
  • mask=('None', 'None')